# Multi-class Classification
In this exercise, we will be using logistic regression and neural networks to recognize handwritten digits (from 0 to 9). First, lets load in our dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import optimize

In [2]:
data = loadmat('ex3data1.mat')
X = data['X']
X = np.insert(X, 0, 1, axis=1) # insert column of 1s into X to account for bias
y = data['y']
y[y == 10] = 0 # replace all of the 10s with 0s
theta = np.zeros(np.size(X, 1))
num_labels = 10 # the number of classes
display(X.shape, y.shape, theta.shape)

(5000, 401)

(5000, 1)

(401,)

If we look at 100 rows from X and map each row to a 20 by 20 pixel grayscale image, we can display some of the images of handwritten numbers that we are working with. The diagram below is taken from the exercise's instruction pdf.

<img src="data_image.png" width=250></img>

Before we begin anything, we are first going to implement a completely vectorized version of logistic regression that does not use any `for` loops. This includes vectorizing the cost function and gradient function, and doing this for regularized logistic regression as well. 

In [3]:
def sigmoid(x):
    return 1/(1 + np.e**(-x))

In [4]:
def cost(theta, X, y):
    m,n = X.shape
    theta = np.reshape(theta, (n,1))
    h = sigmoid(X @ theta)
    h[h == 1] = 0.999 # log(1)=0 causes error in division; credit to Utku Ufuk for this line
    inner = -y*np.log(h)-(1-y)*np.log(1-h)
    return inner.sum()/m

In [5]:
def gradient(theta, X, y): 
    m,n = X.shape
    theta = np.reshape(theta, (n,1))
    h = sigmoid(X @ theta)
    inner = X.T @ (h - y) 
    return (inner/m).flatten()

In [6]:
def costReg(theta, X, y, regParam):
    m,n = X.shape
    theta = np.reshape(theta, (n,1))
    h = sigmoid(X @ theta)
    h[h == 1] = 0.999 # log(1)=0 causes error in division; credit to Utku Ufuk for this line
    inner = -y*np.log(h)-(1-y)*np.log(1-h)
    outer = inner.sum()/m
    reg = regParam/(2*m) * (theta**2).sum()
    return outer + reg

In [7]:
def gradientReg(theta, X, y, regParam):
    m,n = X.shape
    theta = np.reshape(theta, (n,1))
    h = sigmoid(X @ theta)
    inner = X.T @ (h - y)
    outer = inner/m
    reg_theta = theta.copy() 
    reg_theta[0] = 0
    reg = (regParam/m)*reg_theta
    return (outer + reg).flatten()

Now that we have our cost and gradient functions ready, we can start implementing one-vs-all classification. How this works is that we are going to pass in X, y (which is a vector of numbers from 0 to 9), num_labels (the number of classes), and regParam (the regularization parameter). This function will loop through each possible value of an element in y (a number that was drawn) and modify y so that it is a vector of 1s and 0s, a 1 meaning that the element in the original y vector is the same as the current number the loop is focusing on. For example, if the loop is on the value 7, all the 7s in y will be 1 and all the non 7s will be 0. We will then pass this modified y vector into our optimization formula and return an array of n parameters. This array of parameters is optimized to return the probability that the input would be the value of the loop. 

This function will return a matrix of dimension $K \times (N+1)$. 

In [8]:
# returns a matrix where each row corresponds to the learned parameters for each class
# K x (n+1)
def oneVsAll(X, y, num_labels, regParam):
    m,n = X.shape
    classifiers = np.zeros((num_labels, n))
    
    for k in range(num_labels):
        y_class = (y == k).astype(int)
        theta = np.zeros(n) 
        classifiers[k] = optimize.fmin_cg(costReg, theta, fprime=gradientReg, args=(X, y_class, regParam))

    return classifiers

In [9]:
optimal_classifiers = oneVsAll(X, y, 10, 1)

         Current function value: 0.022566
         Iterations: 25
         Function evaluations: 151
         Gradient evaluations: 140
         Current function value: 0.027534
         Iterations: 35
         Function evaluations: 173
         Gradient evaluations: 162
         Current function value: 0.069455
         Iterations: 111
         Function evaluations: 384
         Gradient evaluations: 372
         Current function value: 0.076384
         Iterations: 29
         Function evaluations: 138
         Gradient evaluations: 126
Optimization terminated successfully.
         Current function value: 0.052456
         Iterations: 96
         Function evaluations: 301
         Gradient evaluations: 301
Optimization terminated successfully.
         Current function value: 0.076862
         Iterations: 140
         Function evaluations: 377
         Gradient evaluations: 377
         Current function value: 0.035782
         Iterations: 48
         Function evaluations: 271
     

In [10]:
optimal_classifiers.shape

(10, 401)

Whew, finally we managed to have a set of optimal parameters for each class. Now lets see how accurate our parameters actually are. In order to do so, we first need to predict, for each input of 20x20 pixels, what number our multi class logistic regression would classify the pixels as. After gathering the predicted numbers, we can run them through a function to get the testing accuracy of our multi class logistic regression.

In [11]:
def predict(X, classifiers): 
    probabilities = sigmoid(X @ classifiers.T) # each column is the probability for that class 
    predictions = np.argmax(probabilities, axis=1)
    return np.reshape(predictions, (np.size(predictions, 0),1))

In [12]:
def testingAccuracy(actual, predicted):
    m = np.size(actual, 0)
    numCorrect = 0
    for i in range(m):
        if actual[i][0] == predicted[i][0]:
            numCorrect += 1
    return numCorrect/m

In [13]:
testingAccuracy(y, predict(X, optimal_classifiers))

0.941

When given an regularization parameter of 1, our multi class logistic regression model correctly classifies 94.1% of the numbers. 

# Neural Networks


In this part of the exercise, we will be implementing a neural network to recognize handwritten digits using the same training set as before. Our neural network will have 3 layers: the input layer, a hidden layer, and the output layer. The input layer will have 400 units (and then another one which is the bias unit). We have been provided with a set of network parameters $(\Theta^{(1)}, \Theta^{(2)})$ which have already been trained. These parameters have dimensions that are sized for a neural network with 25 units in the second layer and 10 output units (corresponding to the 10 digit classes).

Let's load in these weights.

In [14]:
nn_weights = loadmat('ex3weights.mat')
theta1 = nn_weights['Theta1']
theta2 = nn_weights['Theta2']
display(X.shape, theta1.shape, theta2.shape)

(5000, 401)

(25, 401)

(10, 26)

In order to implement this neural network, we will be implementing the feedforward propagation method. Below is the implemented function. 

In [15]:
def feedForward(input, theta1, theta2):
    firstLayer = input.T # 401x5000
    secondLayer = sigmoid(theta1 @ firstLayer) # 25x401 * 401x5000 = 25x5000 
    secondLayer = np.insert(secondLayer, 0, 1, axis=0) # 26x5000
    output = sigmoid(theta2 @ secondLayer) # 10x26 * 26x5000 = 10x5000
    return output

In [19]:
h = feedForward(X, theta1, theta2)
h

array([[1.12661530e-04, 4.79026796e-04, 8.85702310e-05, ...,
        5.17641791e-02, 8.30631310e-04, 4.81465717e-05],
       [1.74127856e-03, 2.41495958e-03, 3.24266731e-03, ...,
        3.81715020e-03, 6.22003774e-04, 4.58821829e-04],
       [2.52696959e-03, 3.44755685e-03, 2.55419797e-02, ...,
        2.96297510e-02, 3.14518512e-04, 2.15146201e-05],
       ...,
       [4.01468105e-04, 2.39107046e-03, 6.22892325e-02, ...,
        2.15667361e-03, 1.19366192e-02, 5.73434571e-03],
       [6.48072305e-03, 1.97025086e-03, 5.49803551e-03, ...,
        6.49826950e-01, 9.71410499e-01, 6.96288990e-01],
       [9.95734012e-01, 9.95696931e-01, 9.28008397e-01, ...,
        2.42384687e-05, 2.06173648e-04, 8.18576980e-02]])

The output of this is similar to that of our multi class logistic regression. At each input value, it gives and array of probabilities of the input being that index number. We can rewrite a prediction function that predicts what each handwritten image is and then test the accuracy of it. 

In [17]:
def predict2(h):
    probabilities = h.T
    predictions = np.argmax(probabilities, axis=1)
    predictions = (predictions + 1) % 10 # need this to shift over the results since the weights are assuming that 0 is represented as 10)
    return np.reshape(predictions, (np.size(predictions, 0),1))

In [18]:
testingAccuracy(y, predict2(h))

0.9752

The testing accuracy of our neural network is around 97.5%!!!

This concludes exercise 3.